In [13]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv("WebData.csv")

In [48]:
df

,日期,代號,名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價
0,2020-01-02,1101,台泥,18470566,6251,813465904,43.80,44.15,43.80,44.10
1,2020-01-02,1102,亞泥,8890485,4391,433140140,48.10,49.00,48.05,48.90
2,2020-01-02,1103,嘉泥,2194046,883,49255964,22.40,22.70,22.35,22.35
3,2020-01-02,1104,環泥,867516,384,17026458,19.60,19.70,19.55,19.65
4,2020-01-02,1108,幸福,310216,162,2593989,8.38,8.45,8.28,8.37
...,...,...,...,...,...,...,...,...,...,...
1183580,2024-12-31,9944,新麗,115881,134,2329626,20.05,20.30,19.95,20.30
1183581,2024-12-31,9945,潤泰新,4357585,3703,186985362,43.30,43.35,42.65,42.90
1183582,2024-12-31,9946,三發地產,156969,161,3551688,22.95,22.95,22.50,22.60
1183583,2024-12-31,9955,佳龍,123625,178,3521198,28.70,28.70,28.30,28.50


In [51]:
def add_factor(df , win=5):
    df = df.copy()
    df["日期"] = pd.to_datetime(df["日期"]).dt.normalize()
    
    df['vwap'] = df.iloc[:]["成交金額"]/df.iloc[:]["成交股數"]
    df["sum_value_5d"] = (
    df.groupby("代號")["成交金額"]
      .transform(lambda s: s.shift(1).rolling(5, min_periods=5).sum())
    )
    df["sum_vol_5d"] = (
        df.groupby("代號")["成交股數"]
          .transform(lambda s: s.shift(1).rolling(5, min_periods=5).sum())
    )
    df["vwap_5d"] = df["sum_value_5d"] / df["sum_vol_5d"]
    
    df["open-avg_vwap"] = df["開盤價"] - df["vwap_5d"]
    
    df["close-vwap"] = df["收盤價"] - df["vwap"]
    
    grp = df.groupby("日期")["open-avg_vwap"]
    r = grp.rank(method="average")
    n = grp.transform("size")
    df["cs_rank_open-avg_vwap"] = (r - 1) / (n - 1)
    
    grp = df.groupby("日期")["close-vwap"]
    r = grp.rank(method="average")          
    n = grp.transform("size")
    df["cs_rank_close-vwap"] = (r - 1) / (n - 1)
    
    df["cs_rank_open-avg_vwap*close-vwap"] = ((df["cs_rank_open-avg_vwap"] * df["cs_rank_close-vwap"])-0.5) * -2
    
    df = df.sort_values(['代號','日期'])

    df['std_high_5'] = (
        df.groupby('代號')['最高價']
          .transform(lambda s: s.shift(1).rolling(win, min_periods=win).std())
    )

    df['rank_std_high_5'] = (
        df.groupby('日期')['std_high_5'].rank(pct=True)
    )

    df['corr_hv_5'] = (
        df.groupby('代號', group_keys=False)
          .apply(lambda g: g['最高價'].rolling(win, min_periods=win).corr(g['成交股數']/1000))
          .reset_index(level=0, drop=True).replace([np.inf, -np.inf], 0)
    )
    
    grp = df.groupby("代號", group_keys=False)
    open1 = grp["開盤價"].shift(-1)
    close1 = grp["收盤價"].shift(-1)
    close3 = grp["收盤價"].shift(-3)

    df["ret1d"] = (close1 / open1) - 1
    df["ret3d"] = (close3 / open1) - 1
    
    df = df.drop(columns=["最低價","最高價","成交金額","成交股數","成交筆數","sum_value_5d","sum_vol_5d","vwap","vwap_5d","std_high_5","cs_rank_open-avg_vwap","cs_rank_close-vwap"])
    df = df.dropna()
    
    df = df.sort_values(["日期","代號"], ignore_index=True)
    
    return df

In [52]:
df1 = add_factor(df)

In [53]:
df1

,日期,代號,名稱,開盤價,收盤價,open-avg_vwap,close-vwap,cs_rank_open-avg_vwap*close-vwap,rank_std_high_5,corr_hv_5,ret1d,ret3d
0,2020-01-09,1101,台泥,43.40,43.45,-0.281959,0.034551,0.426299,0.534685,0.068691,0.001151,0.025316
1,2020-01-09,1102,亞泥,48.55,48.95,0.125706,0.170082,-0.522899,0.690502,0.784326,-0.007150,0.001021
2,2020-01-09,1103,嘉泥,21.40,21.55,-0.455489,0.139096,0.473181,0.589114,0.916373,-0.002320,0.004640
3,2020-01-09,1104,環泥,19.05,19.05,-0.378377,-0.004739,0.648784,0.397012,0.968930,-0.002611,-0.002611
4,2020-01-09,1108,幸福,8.15,8.18,-0.157160,0.007959,0.350237,0.115261,0.668492,-0.013317,-0.004843
...,...,...,...,...,...,...,...,...,...,...,...,...
1146045,2024-12-26,9944,新麗,20.10,20.35,0.082803,0.014048,0.596240,0.389158,-0.753492,-0.019608,-0.004902
1146046,2024-12-26,9945,潤泰新,43.45,43.20,0.530196,-0.120362,0.531859,0.209100,-0.218872,-0.003472,-0.006944
1146047,2024-12-26,9946,三發地產,23.70,23.20,0.455357,-0.180938,0.633320,0.363988,0.795507,-0.030303,-0.021645
1146048,2024-12-26,9955,佳龍,28.70,28.75,0.775451,-0.119026,0.470710,0.484027,0.610103,-0.017271,-0.015544


In [54]:
df1.to_csv('withfactor.csv',index = False)